# Taxi Travel Data Analysis

In this demo, we will be doing some demos on temporal feature engineering with the Kaggle Dataset

### Loading libraries, datasets

In [1]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torch

In [2]:
# These are all of the files you are given
#df_tr = pd.read_csv("archive/train.csv")

In [3]:
#df_tr.head()
# avg = df_tr['LEN'].mean()
# print(avg)

### Get Computed Time from POLYLINE

Our goal is to predict the travel-time of the taxi, which can be derived from the POLYLINE length.

Recall:

```
The travel time of the trip (the prediction target of this project) is defined as the (number of points-1) x 15 seconds. 
For example, a trip with 101 data points in POLYLINE has a length of (101-1) * 15 = 1500 seconds. Some trips have missing 
data points in POLYLINE, indicated by MISSING_DATA column, and it is part of the challenge how you utilize this knowledge.
```

We are not doing anything with the MISSING_DATA. It is up to you to find a way to use (or ignore) that information.

In [4]:
# Over every single 
def polyline_to_trip_duration(polyline):
  return max(polyline.count("[") - 2, 0) * 15

# This code creates a new column, "LEN", in our dataframe. The value is
# the (polyline_length - 1) * 15, where polyline_length = count("[") - 1
# df_tr["LEN"] = df_tr["POLYLINE"].apply(polyline_to_trip_duration)

In [5]:
from datetime import datetime
def parse_time(x):
  # We are using python's builtin datetime library
  # https://docs.python.org/3/library/datetime.html#datetime.date.fromtimestamp

  # Each x is essentially a 1 row, 1 column pandas Series
  dt = datetime.fromtimestamp(x["TIMESTAMP"])
  return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

# Because we are assigning multiple values at a time, we need to "expand" our computed (year, month, day, hour, weekday) tuples on 
# the column axis, or axis 1
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html
# df_tr[["YR", "MON", "DAY", "HR", "WK"]] = df_tr[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")

In [6]:
def onehot(x):
    x=x.values[0]
    if x=="A":
        return 1, 0, 0
    elif x=="B":
        return 0, 1, 0
    elif x=="C":
        return 0, 0, 1
    else:
        return 0, 0, 0
    
# df_tr[["CALL_A", "CALL_B", "CALL_C"]] = df_tr[["CALL_TYPE"]].apply(onehot, axis=1, result_type="expand")   
# df_tr[["DAY_A", "DAY_B", "DAY_C"]] = df_tr[["DAY_TYPE"]].apply(onehot, axis=1, result_type="expand")     


### Create a Prediction File

In [7]:
# vals = df_tr["HR"].value_counts()
# print(vals)
# def reduce(x):
#     return x - 2013
# hr_oh = torch.nn.functional.one_hot(torch.tensor(df_tr["HR"].values))
# print(hr_oh)
# print(hr_oh.shape)

In [8]:
# vals = df_tr["TAXI_ID"].values
# print(vals)
# print(min(vals))
# print(max(vals))

# # df_tr["TAXI_ID"] = df_tr["TAXI_ID"].apply(reduce)
# vals = df_tr["TAXI_ID"].values
# print(vals)
# print(min(vals))
# print(max(vals))
# oh = torch.nn.functional.one_hot(torch.tensor(df_tr["TAXI_ID"].values))
# print(oh)
# print(oh.shape)


In [9]:
# df_tr[["CALL_A", "LEN"]].values

In [10]:
# Feature Trimming
# mean, std = df_tr["LEN"].mean(), df_tr["LEN"].std()
# median = df_tr["LEN"].median()
# outlier_threshold = 3
# df_trimmed = df_tr[df_tr["LEN"] < mean + outlier_threshold * std]
# df_trimmed = df_trimmed[df_trimmed['MISSING_DATA'] == False]
# print("Before Trimming: " + str(len(df_tr)))
# print("After Trimming: " + str(len(df_trimmed)))

# df_tr = df_trimmed

In [11]:

class MyDataset(Dataset):
  def __init__(self, csvpath):
    df_tr = pd.read_csv(csvpath)

    if "POLYLINE" not in df_tr: #test dataset
      df_tr["POLYLINE"]="trololololo"
      df_tr["LEN"] = df_tr["POLYLINE"].apply(polyline_to_trip_duration)
    else: 
      df_tr["LEN"] = df_tr["POLYLINE"].apply(polyline_to_trip_duration)
      # first trim the dataset

      mean, std = df_tr["LEN"].mean(), df_tr["LEN"].std()
      median = df_tr["LEN"].median()
      outlier_threshold = 3
      df_trimmed = df_tr[df_tr["LEN"] < mean + outlier_threshold * std]
      df_trimmed = df_trimmed[df_trimmed['MISSING_DATA'] == False]
      print("Before Trimming: " + str(len(df_tr)))
      print("After Trimming: " + str(len(df_trimmed)))

      df_tr = df_trimmed


    df_tr[["YR", "MON", "DAY", "HR", "WK"]] = df_tr[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")
    df_tr[["CALL_A", "CALL_B", "CALL_C"]] = df_tr[["CALL_TYPE"]].apply(onehot, axis=1, result_type="expand")   
    df_tr[["DAY_A", "DAY_B", "DAY_C"]] = df_tr[["DAY_TYPE"]].apply(onehot, axis=1, result_type="expand")
    hr_oh = torch.nn.functional.one_hot(torch.tensor(df_tr["HR"].values))
    
         
    x=df_tr[["CALL_A", "CALL_B", "CALL_C", "DAY_A", "DAY_B", "DAY_C"]].values
    
    y=df_tr["LEN"].values
 
    self.x_train=torch.tensor(x,dtype=torch.float32)
    self.x_train=torch.cat([self.x_train, hr_oh], dim=1)
    print(self.x_train)
    self.y_train=torch.tensor(y,dtype=torch.float32)
    self.df = df_tr
 
  def __len__(self):
    return len(self.y_train)
   
  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]

In [23]:
myDs=MyDataset("archive/train.csv")
train_loader=DataLoader(myDs,batch_size=64, shuffle=True)

Before Trimming: 1710670
After Trimming: 1692763
tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.]])


In [36]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten(start_dim=1)
        # self.conv1 = nn.Conv2d(3, 6, 5)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = nn.Conv2d(6, 16, 5)
        self.dro = nn.Dropout(p=0.3)
        self.fc1 = nn.Linear(30, 1000)
        self.fc2 = nn.Linear(1000, 1000)
        self.fc3 = nn.Linear(1000, 1000)
        self.fc4 = nn.Linear(1000, 1000)
        self.fc5 = nn.Linear(1000, 1000)
        self.fc6 = nn.Linear(1000, 1000)
        self.fc7 = nn.Linear(1000, 1)

    def forward(self, x):
        # x = self.pool(F.relu(self.conv1(x)))
        # x = self.pool(F.relu(self.conv2(x)))
        # x = torch.flatten(x, -1) # flatten all dimensions except batch
        
        x = F.relu(self.fc1(x))
        x = self.dro(x)
        x = F.relu(self.fc2(x))
        x = self.dro(x)
        x = F.relu(self.fc3(x))
        x = self.dro(x)
        x = F.relu(self.fc4(x))
        x = self.dro(x)
        x = F.relu(self.fc5(x))
        x = self.dro(x)
        x = F.relu(self.fc6(x))
        x = self.dro(x)
        x = self.fc7(x)
        # x = torch.transpose(x, 0, 1)
        return x


net = Net()

In [37]:
import torch.optim as optim
def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))
criterion = RMSELoss
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

In [38]:
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        labels=labels.unsqueeze(1)
        outputs = net(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print(outputs[0], labels[0])
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

tensor([0.0748], grad_fn=<SelectBackward0>) tensor([660.])
[1,   100] loss: 774.387
tensor([0.2049], grad_fn=<SelectBackward0>) tensor([315.])
[1,   200] loss: 786.993
tensor([0.3354], grad_fn=<SelectBackward0>) tensor([1020.])
[1,   300] loss: 781.526
tensor([0.4569], grad_fn=<SelectBackward0>) tensor([795.])
[1,   400] loss: 780.316
tensor([0.6175], grad_fn=<SelectBackward0>) tensor([960.])
[1,   500] loss: 774.131
tensor([0.7467], grad_fn=<SelectBackward0>) tensor([780.])
[1,   600] loss: 786.163
tensor([0.9166], grad_fn=<SelectBackward0>) tensor([465.])
[1,   700] loss: 778.604
tensor([1.1427], grad_fn=<SelectBackward0>) tensor([60.])
[1,   800] loss: 781.901
tensor([1.3325], grad_fn=<SelectBackward0>) tensor([1155.])
[1,   900] loss: 783.279
tensor([1.7277], grad_fn=<SelectBackward0>) tensor([540.])
[1,  1000] loss: 771.950
tensor([2.0260], grad_fn=<SelectBackward0>) tensor([615.])
[1,  1100] loss: 783.889
tensor([2.4849], grad_fn=<SelectBackward0>) tensor([1035.])
[1,  1200] loss

In [ ]:
# zero_call = df_tr.sort_values(by='ORIGIN_CALL', ascending=True)
# print(zero_call)

In [ ]:
testset = MyDataset("archive/test_public.csv")
testloader = DataLoader(testset)
preds = []
with torch.no_grad():
    for data in testloader:
        features, labels = data
        # calculate outputs by running images through the network
        output = net(features)
        preds.append(round(output.item()))
        # the class with the highest energy is what we choose as prediction
ids = testset.df["TRIP_ID"]        
print(preds)
d = {"TRIP_ID" : ids, "TRAVEL_TIME" : preds}
newdf = pd.DataFrame(d)
print(newdf)
newdf.to_csv("my_pred.csv", index=None)

tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])
[748, 769, 800, 784, 838, 830, 842, 812, 767, 760, 794, 837, 809, 831, 880, 807, 805, 712, 817, 791, 806, 857, 828, 840, 746, 844, 779, 846, 768, 793, 743, 752, 794, 819, 793, 817, 838, 828, 776, 848, 771, 805, 795, 746, 757, 793, 854, 854, 767, 837, 835, 831, 743, 766, 800, 751, 795, 786, 885, 849, 869, 826, 794, 770, 764, 762, 818, 869, 809, 810, 759, 860, 742, 793, 665, 745, 682, 754, 738, 713, 773, 658, 741, 711, 709, 721, 727, 724, 624, 813, 735, 719, 700, 697, 682, 816, 744, 727, 743, 751, 641, 711, 749, 718, 737, 737, 706, 687, 692, 703, 772, 763, 718, 778, 685, 708, 640, 667, 695, 661, 714, 768, 719, 713, 720, 723, 753, 712, 745, 662, 672, 652, 741, 688, 743, 703, 680, 668, 735, 735, 765, 690, 779, 662, 756, 712, 623, 741, 697, 

In [ ]:
# # Sample submission file that is given on kaggle
# df_sample = pd.read_csv("archive/sampleSubmission.csv")

# df_sample["TRAVEL_TIME"] = 716.43

# # mean(716.43) -> 792.73593
# # median(600) -> 784.74219
# df_sample.to_csv("my_pred.csv", index=None)

### Do some Feature Analysis

For our feature analysis, we are looking at which of our engineered features may be useful in making a taxicab time regression model

In [ ]:
# First n samples to analyze. Set to -1 to use all data
end = -1

outlier_threshold = 3

# "Choose all data, where the trip length is less than 3 standard deviations away from the mean"
# This is to remove outliers. Otherwise, our plots would look very squished (since there are some
# VERRRRRY long taxi trips in the dataset)
df_trimmed = df_tr[df_tr["LEN"] < mean + outlier_threshold * std]

# Because our y-values only take on multiples of 15, we want just enough buckets in a histogram
# such that each buckets counts one value's frequency. (e.x. one bucket counts how many 15s trips, 
# how many 30s trips, etc. )
buckets = (int(mean + outlier_threshold * std) // 15)

print(f"Using: {len(df_trimmed)}/{len(df_tr)}")

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(18,14))

# Now, we visualize some features that we think might be useful
for idx, v in enumerate(["YR", "MON", "DAY", "HR", "WK", "ORIGIN_STAND"]):
  # idx // 3 = row, idx % 3 = column
  ax = axs[idx // 3, idx % 3]
  
  # Remove any rows with invalid values
  df_subset = df_trimmed.dropna(subset=v)
  
  # Create a histogram. Look up the documentation for more details
  ax.hist2d(df_subset[v][:end], df_subset["LEN"][:end], cmap="CMRmap", bins=(120,buckets))
  
  # Some stylistic things to make the graphs look nice
  ax.set_xlim(ax.get_xlim()[0] - 1, ax.get_xlim()[1] + 1)
  ax.set_facecolor("black")
  ax.set_ylabel("seconds", fontsize=18)
  ax.set_title(f"Feature: {v}", fontsize=20)


NameError: name 'df_tr' is not defined

In [ ]:
plt.figure(figsize=(10,10))
for v in [0, 5, 11, 17, 23]:
  # Filter data where the HR matches v
  hourly_data = df_trimmed[df_trimmed["HR"] == v]["LEN"]
  histogram, bin_boundary = np.histogram(hourly_data, bins=buckets)
  histogram = histogram / len(hourly_data)
  # The center is the left_bound and right_bound of a bucket
  bin_centers = [(bin_boundary[i] + bin_boundary[i + 1]) / 2 for i in range(buckets)]
  plt.plot(bin_centers, histogram, label=f"HR={v}")
plt.legend();